In [ ]:
%run 'pg_config.py'
import json, time, traceback
import requests as rq
from sqlalchemy import create_engine, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.dialects.postgresql import insert

In [ ]:
engine = create_engine('postgresql://' + PG_USER + ':' + PG_PASSWORD + '@localhost/' + PG_DATABASE)
session = sessionmaker(bind=engine)

In [ ]:
import helpdesk_model as hm

In [ ]:
def get_departments(api_key=API_KEY, endpoint=ENDPOINT):
    data = rq.get(endpoint + 'departments?apikey=' + api_key).json()
    with engine.begin() as con:
        vals = [
            {
                'id' : d['departmentid'],
                 'name' : d['name'],
                 'description' : d['description'],
                 'preset_status' : d['presetstatus']
            } for d in data['response']['departments']]
        stm = insert(hm.Department).on_conflict_do_nothing()
        con.execute(stm, vals)
get_departments()

In [ ]:
def get_tags(api_key=API_KEY, endpoint=ENDPOINT):
    data = rq.get(endpoint + 'tags?apikey=' + api_key).json()
    with engine.begin() as con:
        vals = [{'id' : d['id'], 'name' : d['name']} for d in data['response']['tags']]
        stm = insert(hm.Tag).on_conflict_do_nothing()
        con.execute(stm, vals)
get_tags()

In [ ]:
def get_customers_fetcher(limit, offset, max=None, api_key=API_KEY, endpoint=ENDPOINT):
    def create_url(offset):
        url = endpoint + 'customers' \
        + '?apikey=' + api_key \
        + '&limitcount=' + str(limit) \
        + '&limitfrom=' + str(offset)
        return url
    
    while True:
        if max is not None:
            if offset >= max:
                break
        data = rq.get(create_url(offset)).json()
        if not data['response']['customers']:
            break
        
        yield data['response']['customers']
        
        offset += limit
                
def get_customers(offset, limit=1000, max=None):
    stm = insert(hm.Customer).on_conflict_do_nothing()
    for i in get_customers_fetcher(limit, offset, max):
        with engine.begin() as con:
            vals = [
                {
                    'contact_id' : c['contactid'],
                    'user_id' : c['userid'],
                    'date_created' : c['datecreated'],
                    'role' : c['role'],
                    'gender' : c['gender'],
                    'email' : c['email']
                } for c in i]
            for j in vals:
                con.execute(stm,j)
        time.sleep(2)
        
    return True
        
while True:
    customers_count = session().query(func.count(hm.Customer.contact_id)).first()[0]
    try:
        if get_customers(customers_count):
            break
    except Exception:
        traceback.print_exc()
        time.sleep(20)

In [ ]:
def get_conversations_fetcher(limit, offset, max=None, api_key=API_KEY, endpoint=ENDPOINT):
    def create_url(offset):
        url = endpoint + 'conversations' \
            + '?apikey=' + api_key \
            + '&limit=' + str(limit) \
            + '&offset=' + str(offset)
        return url

    while(True):
        if max is not None:
            if offset >= max:
                break

        data = rq.get(create_url(offset)).json()
        if not data['response']['conversations']:
            break

        yield data['response']['conversations']

        offset += limit

def get_messages_fetcher(conv_vals, api_key=API_KEY, endpoint=ENDPOINT):
    def create_url(conv_id):
        url = endpoint + \
            + 'conversations/' + conv_id + '/messages?apikey=' + api_key
        return url
        
    for i in conv_vals:
        data = rq.get(create_url(i['id'])).json()
        yield((i['id'], data['response']))
            
def get_conversations(offset, limit=1000, max=None):
    def get_message_groups(conv_vals):
        
        mgrp_vals = []
        msgs_vals = []
        
        for id, data_global in get_messages_fetcher(conv_vals):
            for data in data_global['groups']:
                mgrp_vals.append(
                    {
                        'id' : data['messagegroupid'],
                        'user_id' : data['userid'],
                        'rtype' : data['rtype'],
                        'rstatus' : data['rstatus'],
                        'date_created' : data['datecreated'],
                        'date_finished' : data['datefinished'],
                        'conversation_id' : id
                    })
                
                data_msgs = []
                if 'messages' in data:
                    data_msgs = data['messages']
                
                for dt in data_msgs:
                    msgs_vals.append(
                    {
                        'id' : dt['messageid'],
                        'user_id' : dt['userid'],
                        'rtype' : dt['rtype'],
                        'date_created' : dt['datecreated'],
                        'message_text' : dt['message'].replace('\x00', ''), #psycopg2 disallows NUL
                        'message_group_id' : data['messagegroupid']
                    })
        
        return mgrp_vals, msgs_vals

    conv_stm = insert(hm.Conversation).on_conflict_do_nothing()
    tags_stm = insert(hm.tags_conversations).on_conflict_do_nothing()
    mgrp_stm = insert(hm.MessageGroup).on_conflict_do_nothing()
    msgs_stm = insert(hm.Message).on_conflict_do_nothing()
    for i in get_conversations_fetcher(limit, offset, max):
        time.sleep(2)
        conv_vals = []
        tags_vals = []

        for c in i:
            conv_vals.append(
                 {
                    'id' : c['conversationid'],
                    'department_id' : c['departmentid'],
                    'status' : c['status'],
                    'channel_type' : c['channel_type'],
                    'date_created' : c['datecreated'],
                    'date_changed' : c['datechanged'],
                    'date_due' : c['datedue'],
                    'owner_name' : c['ownername'],
                    'owner_email' : c['owneremail'],
                    'subject' : c['subject'],
                })
            tags = c['tags'].split(',') if c['tags'] is not None else None
            if tags is not None:
                tags_vals += [{
                    'conversation_id' : c['conversationid'],
                    'tag_id' : t} for t in tags]
            
        mgrp_vals, msgs_vals = get_message_groups(conv_vals)

        #print(conv_vals)
        #print(tags_vals)
        #print(mgrp_vals)
        #print(msgs_vals)
        with engine.begin() as con:
            if len(conv_vals) > 0:
                con.execute(conv_stm, conv_vals)
            if len(tags_vals) > 0:
                con.execute(tags_stm, tags_vals)
            if len(mgrp_vals) > 0:
                con.execute(mgrp_stm, mgrp_vals)
            if len(msgs_vals) > 0:                
                con.execute(msgs_stm, msgs_vals)
        
    return True

while True:
    conversations_count = session().query(func.count(hm.Conversation.id)).first()[0]
    try:
        if get_conversations(conversations_count, limit=500):
            break
    except Exception:
        traceback.print_exc()
        time.sleep(20)